Nama :** Rully Lukas T      **                                                                                               
NIM : **1103200181    **                                                                                                                                             
Model : **SplitOVer**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


\

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from shutil import copyfile
import os.path
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array,load_img
from PIL import Image
from sklearn.model_selection import train_test_split
from numpy import load
import matplotlib.pyplot as plt
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
imagegen = ImageDataGenerator()

In [ ]:
patho = {'Bayam': 0, 'Kangkung_Air': 1, 'Kangkung_Darat': 2, 'Kelor': 3, 'Pakis': 4}

In [ ]:
# Load train data from drive
train_generator = imagegen.flow_from_directory("/content/drive/MyDrive/Statistik",
                                               class_mode="categorical",
                                               shuffle=False,
                                               batch_size=16,  # Use a smaller batch size
                                               target_size=(512, 512),
                                               seed=42)

x_list = []
y_list = []

Found 1575 images belonging to 5 classes.


In [ ]:
# Load and process images in batches
for i in range(len(train_generator)):
    x_batch, y_batch = train_generator.next()
    x_list.append(x_batch)
    y_list.append(y_batch)

    # Flatten labels and check unique classes
    y_labels = np.argmax(y_batch, axis=1)
    unique_classes = np.unique(y_labels)

    # Check if we have at least two classes and enough samples for ADASYN
    if len(unique_classes) > 1:
        x_concat = np.concatenate(x_list)
        y_concat = np.concatenate(y_list)

        X_batch = x_concat.reshape(x_concat.shape[0], -1)
        y_labels_concat = np.argmax(y_concat, axis=1)

        # Set the n_neighbors parameter to be less than or equal to the number of samples in the smallest class
        min_samples = np.min(np.bincount(y_labels_concat))
        n_neighbors = max(1, min(5, min_samples - 1))

        if min_samples > 1:  # Ensure we have more than one sample for the smallest class
            adasyn = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbors, random_state=42)
            X_adasyn, y_adasyn = adasyn.fit_resample(X_batch, y_labels_concat)
            X_adasyn = X_adasyn.reshape(-1, 512, 512, 3)

            # Save ADASYN generated images
            train_sep_dir = '/content/drive/MyDrive/Bukti_kas'

            # Create a "testfolder" if it does not exist on the drive
            if not os.path.exists(train_sep_dir):
                os.mkdir(train_sep_dir)

            # This function returns the label name
            def get_key(val):
                for key, value in patho.items():
                    if val == value:
                        return key

            for j in range(len(X_adasyn)):
                label = get_key(y_adasyn[j])
                label_dir = os.path.join(train_sep_dir, str(label))

                if not os.path.exists(label_dir):
                    os.mkdir(label_dir)

                pil_img = array_to_img(X_adasyn[j] * 255)
                pil_img.save(os.path.join(label_dir, 'adasyn_' + str(i) + '_' + str(j) + '.jpg'))

            # Reset lists after processing
            x_list = []
            y_list = []

            print(f"Processed batch {i+1}/{len(train_generator)}")

# If there's any leftover data that wasn't processed, process it now
if len(x_list) > 0:
    x_concat = np.concatenate(x_list)
    y_concat = np.concatenate(y_list)

    X_batch = x_concat.reshape(x_concat.shape[0], -1)
    y_labels_concat = np.argmax(y_concat, axis=1)

    min_samples = np.min(np.bincount(y_labels_concat))
    n_neighbors = max(1, min(5, min_samples - 1))

    if min_samples > 1:
        adasyn = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbors, random_state=42)
        X_adasyn, y_adasyn = adasyn.fit_resample(X_batch, y_labels_concat)
        X_adasyn = X_adasyn.reshape(-1, 512, 512, 3)

        for j in range(len(X_adasyn)):
            label = get_key(y_adasyn[j])
            label_dir = os.path.join(train_sep_dir, str(label))

            if not os.path.exists(label_dir):
                os.mkdir(label_dir)

            pil_img = array_to_img(X_adasyn[j] * 255)
            pil_img.save(os.path.join(label_dir, 'adasyn_' + 'final_' + str(j) + '.jpg'))

print("Processing complete.")

KeyboardInterrupt: 

In [ ]:
# Load and process images in batches
for i in range(len(train_generator)):
    x_batch, y_batch = train_generator.next()
    x_list.append(x_batch)
    y_list.append(y_batch)

x_concat = np.concatenate(x_list)
y_concat = np.concatenate(y_list)

# Flatten labels
y_labels_concat = np.argmax(y_concat, axis=1)

# Reshape data for undersampling
X_batch = x_concat.reshape(x_concat.shape[0], -1)

# Apply RandomUnderSampler to balance all classes
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_batch, y_labels_concat)

# Reshape back to image dimensions
X_rus = X_rus.reshape(-1, 512, 512, 3)

# Save RandomUnderSampler generated images
train_sep_dir = '/content/drive/MyDrive/Scracth'

# Create a "testfolder" if it does not exist on the drive
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)

# This function returns the label name
def get_key(val):
    for key, value in patho.items():
        if val == value:
            return key

for j in range(len(X_rus)):
    label = get_key(y_rus[j])
    label_dir = os.path.join(train_sep_dir, str(label))

    if not os.path.exists(label_dir):
        os.mkdir(label_dir)

    pil_img = array_to_img(X_rus[j] * 255)
    pil_img.save(os.path.join(label_dir, 'rus_' + str(j) + '.jpg'))

print("Processing complete.")

In [ ]:
/content/drive/MyDrive/Bukti_kas

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths
data_dir = '/content/drive/MyDrive/Statistik/Over'  # Replace with the path to your dataset folder
train_dir = '/content/drive/MyDrive/Statistik/SplitOver/train'   # Replace with the path to save training data
test_dir = '/content/drive/MyDrive/Statistik/SplitOver/test'     # Replace with the path to save testing data

# Set parameters
img_height = 224  # Set to the height of your images
img_width = 224   # Set to the width of your images
batch_size = 32   # You can adjust the batch size according to your requirements

# Define ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(validation_split=0.2)  # 20% for validation

# Load the training data
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Load the validation data
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Optional: Save the split datasets if required
# Here we show how to save them, but it's not necessary if you directly load them as above

import os
import shutil
from tqdm import tqdm

def copy_files(generator, dest_dir):
    class_indices = generator.class_indices
    for class_name, class_index in class_indices.items():
        os.makedirs(os.path.join(dest_dir, class_name), exist_ok=True)

    for i in range(len(generator)):
        x, y = next(generator)
        for j in range(len(x)):
            img = tf.keras.preprocessing.image.array_to_img(x[j])
            label = list(class_indices.keys())[list(y[j]).index(1)]
            img.save(os.path.join(dest_dir, label, f'{i}_{j}.jpg'))

copy_files(train_generator, train_dir)
copy_files(validation_generator, test_dir)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths
data_dir = '/content/drive/MyDrive/Statistik/Over'  # Replace with the path to your dataset folder
train_dir = '/content/drive/MyDrive/Statistik/SplitOver/train'   # Replace with the path to save training data
test_dir = '//content/drive/MyDrive/Statistik/SplitOver/test'     # Replace with the path to save testing data

# Set parameters
img_height = 224  # Set to the height of your images
img_width = 224   # Set to the width of your images
batch_size = 32   # You can adjust the batch size according to your requirements

# Define ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(validation_split=0.2)  # 20% for validation

# Load the training data
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Load the validation data
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Optional: Save the split datasets if required
# Here we show how to save them, but it's not necessary if you directly load them as above

import os
import shutil
from tqdm import tqdm

def copy_files(generator, dest_dir):
    class_indices = generator.class_indices
    for class_name, class_index in class_indices.items():
        os.makedirs(os.path.join(dest_dir, class_name), exist_ok=True)

    for i in range(len(generator)):
        x, y = next(generator)
        for j in range(len(x)):
            img = tf.keras.preprocessing.image.array_to_img(x[j])
            label = list(class_indices.keys())[list(y[j]).index(1)]
            img.save(os.path.join(dest_dir, label, f'{i}_{j}.jpg'))

copy_files(train_generator, train_dir)
copy_files(validation_generator, test_dir)

Found 580 images belonging to 5 classes.
Found 145 images belonging to 5 classes.
